# Food Recognition and Calorie Estimation Model

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [3]:
# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

### Preprocess the Data

In [4]:
# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    'D:/data/food-101/images',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'D:\\data\\food-101\\images',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


### Train the Model

In [5]:
# Define the model architecture
num_classes = len(train_generator.class_indices)  # Dynamically get the number of classes
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Use num_classes for the output layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
EPOCHS = 2
model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)

# Save the trained model
model.save('food_recognition_model.h5')

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/2


C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2525/2525 ━━━━━━━━━━━━━━━━━━━━ 2212s 874ms/step - accuracy: 0.0097 - loss: 4.6174 - val_accuracy: 0.0099 - val_loss: 4.6152
Epoch 2/2
2525/2525 ━━━━━━━━━━━━━━━━━━━━ 1485s 587ms/step - accuracy: 0.0093 - loss: 4.6156 - val_accuracy: 0.0099 - val_loss: 4.6152


In [6]:
# Load the trained model
model = load_model('food_recognition_model.h5')

In [7]:
# Function to preprocess input image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array / 255.

In [8]:
# Function to predict food item and estimate calorie content
def predict_calories(img_path):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    class_labels = list(train_generator.class_indices.keys())
    predicted_label = class_labels[predicted_class]
    return predicted_label, predictions[0][predicted_class]### Make Predictions or Recognitions

### Make Predictions or Recognitions

In [9]:
# Example usage
image_path = '/path_to_image'
food_class, confidence = predict_calories(image_path)
print(f'Predicted Food Class: {food_class}, Confidence: {confidence}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
Predicted Food Class: fried_calamari, Confidence: 0.0101777957752347
